In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family']=['DFKai-SB']
%matplotlib inline

In [2]:
def is_number(s):
    try:
        float(s)
        return float(s)
    except ValueError:
        return s

In [3]:
station = ''
def plot_time_choose(values, sta_now):
    global station
    global datas
    if sta_now != station:
        BaseURL = 'https://www.cwb.gov.tw/V7/observe/24real/Data/'+str(sta_now)+'.htm'
        res = requests.get(BaseURL)
        res.encoding = 'utf-8'
        #print("encoding: %s" % res.encoding)
        A = res.text
        soup = BeautifulSoup(A,'html.parser')
        srcs = soup.select("table.BoxTable > tr")
        col_names = [i.get_text() for i in srcs[0].select('th')]
        datas = pd.DataFrame(np.zeros((len(srcs)-1, len(col_names))), columns=col_names)
        date_index = []

        for index, row in enumerate(srcs[len(srcs):0:-1]):
            for jindex, col in enumerate(row):

                datas.iloc[index, jindex] = np.nan if is_number(col.get_text()) == '-' else is_number(col.get_text())

        datas = datas.set_index(['觀測時間'])
        #datas = datas.set_index(pd.to_datetime(datas.index, format='%m/%d %H:%M'))
        station = sta_now
        
    ## Start to plot
    aa = datas[['溫度(°C>>°F)', '相對溼度(%)', '當日累積雨量(毫米)']]
    start = values[0]
    end = values[1]
    print(start, end)
    try:
        cc = aa.iloc[int(start):int(end)]
        cc.plot(subplots=True, figsize=(10, 6))
        plt.xticks(rotation=45)
        plt.show()
    except:
        print('end is smaller than start or equal to start, or no numerical to plot')

In [4]:
import ipywidgets as widgets
widgets.interact(plot_time_choose,
    values = widgets.IntRangeSlider(value=[0, 150], min=0, max=150, step=1,
    description='Range:', disabled=False, continuous_update=True, orientation='horizontal'),
                 
    sta_now = widgets.Dropdown(options={'鞍部': '46691', '台北': '46692', '信義': 'C0AC7', '台中': '46749', '高雄': '46744'}, 
                               value='46692', description='測站:', disabled=False)
)

<function __main__.plot_time_choose>